# Crawling PTA

In [19]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re
import concurrent.futures

# MAKSIMAL PEKERJA SIMULTAN (BISA DISESUAIKAN)
MAX_WORKERS = 10

def get_fakultas_prodi_list():
    # ... (fungsi ini tidak berubah)
    prodi_list = []
    try:
        url_nav = "https://pta.trunojoyo.ac.id/c_search/byfac"
        r = requests.get(url_nav, timeout=10)
        r.raise_for_status()
        soup = BeautifulSoup(r.content, "html.parser")
        sidebar_nav = soup.select_one('div.box.sidebar_nav')
        if not sidebar_nav: return []
        fakultas_items = sidebar_nav.select_one('ul').find_all('li', recursive=False)
        for item_fakultas in fakultas_items:
            anchor_fakultas = item_fakultas.find('a', recursive=False)
            if not anchor_fakultas: continue
            nama_fakultas = anchor_fakultas.get_text(strip=True)
            ul_prodi = item_fakultas.find('ul')
            if not ul_prodi: continue
            for link_prodi in ul_prodi.select('li a'):
                nama_prodi = link_prodi.get_text(strip=True)
                href = link_prodi.get('href')
                prodi_id = href.strip('/').split('/')[-1]
                if prodi_id.isdigit():
                    prodi_list.append({
                        "id_prodi": int(prodi_id),
                        "nama_prodi": nama_prodi,
                        "nama_fakultas": nama_fakultas
                    })
    except requests.exceptions.RequestException as e:
        print(f"Gagal mengambil daftar prodi: {e}")
    return prodi_list

def scrape_jurnal_detail(jurnal_url):
    # ... (fungsi ini tidak berubah)
    try:
        response = requests.get(jurnal_url, timeout=15)
        response.raise_for_status()
        isi = BeautifulSoup(response.content, "html.parser").select_one('div#content_journal')
        if not isi: return None
        judul = isi.select_one('a.title').text.strip()
        penulis = isi.select_one('span:contains("Penulis")').text.split(' : ')[1].strip()
        pembimbing_pertama = isi.select_one('span:contains("Dosen Pembimbing I")').text.split(' : ')[1].strip()
        pembimbing_kedua = isi.select_one('span:contains("Dosen Pembimbing II")').text.split(':')[1].strip()
        abstract_paragraphs = isi.select('p[align="justify"]')
        text_indo_mentah = abstract_paragraphs[0].text if len(abstract_paragraphs) > 0 else ""
        abstrak_indonesia = re.sub(r'\s+', ' ', text_indo_mentah).strip()
        text_inggris_mentah = abstract_paragraphs[1].text if len(abstract_paragraphs) > 1 else ""
        abstrak_inggris = re.sub(r'\s+', ' ', text_inggris_mentah).strip()
        return {
            "penulis": penulis, "judul": judul, "pembimbing_pertama": pembimbing_pertama,
            "pembimbing_kedua": pembimbing_kedua, "abstrak": abstrak_indonesia,
            "abstrak_inggris": abstrak_inggris
        }
    except requests.exceptions.RequestException:
        return None

def scrape_prodi(prodi):
    # ... (fungsi ini tidak berubah, kecuali pada bagian print)
    jurnal_urls = []
    page = 1
    while len(jurnal_urls) < 4:
        try:
            url = f"https://pta.trunojoyo.ac.id/c_search/byprod/{prodi['id_prodi']}/{page}"
            r = requests.get(url, timeout=10)
            r.raise_for_status()
            soup = BeautifulSoup(r.content, "html.parser")
            jurnals_on_page = soup.select('li[data-cat="#luxury"] a.gray.button')
            if not jurnals_on_page: break
            for a_tag in jurnals_on_page:
                if len(jurnal_urls) < 4:
                    jurnal_urls.append(a_tag['href'])
                else: break
            page += 1
        except requests.exceptions.RequestException:
            break

    if not jurnal_urls:
        # Tambahkan \n di awal print agar lebih rapi
        print(f"\n✔️ Selesai: {prodi['nama_fakultas']} - {prodi['nama_prodi']} | Ditemukan 0 jurnal.")
        return [{
            'id_prodi': prodi['id_prodi'], 'nama_prodi': prodi['nama_prodi'],
            'nama_fakultas': prodi['nama_fakultas'], 'judul': 'Tidak ada jurnal',
            'penulis': None, 'pembimbing_pertama': None, 'pembimbing_kedua': None,
            'abstrak': None, 'abstrak_inggris': None
        }]

    all_jurnal_data = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        hasil_scrape = executor.map(scrape_jurnal_detail, jurnal_urls)
        for hasil in hasil_scrape:
            if hasil:
                hasil['id_prodi'] = prodi['id_prodi']
                hasil['nama_prodi'] = prodi['nama_prodi']
                hasil['nama_fakultas'] = prodi['nama_fakultas']
                all_jurnal_data.append(hasil)
    
    # Tambahkan \n di awal print agar lebih rapi
    print(f"\n✔️ Selesai: {prodi['nama_fakultas']} - {prodi['nama_prodi']} | Berhasil mengambil {len(all_jurnal_data)} jurnal.")
    return all_jurnal_data

def main():
    # ... (fungsi ini tidak berubah)
    start_time = time.time()
    daftar_prodi = get_fakultas_prodi_list()
    if not daftar_prodi: return pd.DataFrame()

    final_results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        results_per_prodi = executor.map(scrape_prodi, daftar_prodi)
        for list_jurnal in results_per_prodi:
            final_results.extend(list_jurnal)

    df = pd.DataFrame(final_results)
    if not df.empty:
        # Mengurutkan DataFrame berdasarkan urutan prodi asli
        original_prodi_order = [p['id_prodi'] for p in daftar_prodi]
        df['id_prodi'] = pd.Categorical(df['id_prodi'], categories=original_prodi_order, ordered=True)
        df = df.sort_values('id_prodi').reset_index(drop=True)
        
        df = df[['nama_fakultas', 'id_prodi', 'nama_prodi', 'judul', 'penulis', 'pembimbing_pertama', 'pembimbing_kedua', 'abstrak', 'abstrak_inggris']]
    
    df.to_csv("pta_final_concurrent_sorted.csv", index=False)
    
    end_time = time.time()
    print("\n\n✅ Proses scraping selesai.") # Tambah baris baru agar rapi
    print(f"Total baris data yang dihasilkan: {len(df)}.")
    print(f"Total waktu eksekusi: {end_time - start_time:.2f} detik.")
    
    return df

# if __name__ == "__main__":
#     df_hasil_cepat = main()
#     df_hasil_cepat

In [20]:
main()


✔️ Selesai: Hukum - Magister Ilmu Hukum | Berhasil mengambil 4 jurnal.

✔️ Selesai: Pertanian - Teknologi Industri Pertanian | Berhasil mengambil 4 jurnal.

✔️ Selesai: Ekonomi Dan Bisnis - Ekonomi Pembangunan | Berhasil mengambil 4 jurnal.

✔️ Selesai: Pertanian - Agroteknologi | Berhasil mengambil 4 jurnal.

✔️ Selesai: Pertanian - Ilmu Kelautan | Berhasil mengambil 4 jurnal.

✔️ Selesai: Pertanian - Agribisnis | Berhasil mengambil 4 jurnal.

✔️ Selesai: Pertanian - Manajemen Sumberdaya Perairan | Berhasil mengambil 4 jurnal.

✔️ Selesai: Hukum - Ilmu Hukum | Berhasil mengambil 4 jurnal.

✔️ Selesai: Ekonomi Dan Bisnis - Manajemen | Berhasil mengambil 4 jurnal.

✔️ Selesai: Pertanian - Magister Pengelolaan Sumber Daya Alam | Berhasil mengambil 3 jurnal.

✔️ Selesai: Ekonomi Dan Bisnis - Doktor Ilmu Manajemen | Ditemukan 0 jurnal.

✔️ Selesai: Ekonomi Dan Bisnis - Akuntansi | Berhasil mengambil 4 jurnal.

✔️ Selesai: Ekonomi Dan Bisnis - D3 Akuntansi | Berhasil mengambil 4 jurnal.

✔

,nama_fakultas,id_prodi,nama_prodi,judul,penulis,pembimbing_pertama,pembimbing_kedua,abstrak,abstrak_inggris
0,Hukum,1,Ilmu Hukum,Implementasi Fungsi Legislasi Dewan Perwakilan...,Dyah Ayu Citra Seza,"Yudi Widagdo Harimurti, SH., MH","Safi', SH., MH",ABSTRAK Implementasi Fungsi Legislasi DPRD Kab...,ABSTRACT Implementation of Legislation Parliam...
1,Hukum,1,Ilmu Hukum,Pertanggungjawaban Pidana Direksi BUMN (Perser...,Maulina Nurlaily,"Tolib Effendi, SH., MH.","Dr. Eni Suastuti, SH., Mhum.",Badan Usaha Milik Negara (BUMN) adalah Badan u...,State Owned Enterprises (SOEs) are business en...
2,Hukum,1,Ilmu Hukum,Analisis Terhadap Kekosongan Hukum dalam Penga...,Moh. Samsul Hidayat,"Tolib Effendi, SH., MH.","Agus Ramdlany, SH., MH.",Kasus narkoba tidak henti-hentinya terdengar d...,"Drug cases endlessly heard on television, radi..."
3,Hukum,1,Ilmu Hukum,PERLINDUNGAN HUKUM BAGI KONSUMEN ATAS PRODUK E...,TOMMY ADITYA PARLINDUNGAN MARBUN,"DR. DJULAEKA, S.H., M.HUM","DR.USWATUN HASANAH, S.H., M. HUM",Produk elektronik adalah suatu benda bergerak ...,Electronic products is an object moves through...
4,Hukum,24,Magister Ilmu Hukum,PERTANGGUNG JAWABAN HUKUM PEJABAT PEMBUAT KOMI...,MOCHAMAD SAICHU,"Dr. Nunuk Nuswardani S.H., M.H","Dr. Deni SB Yuherawan S.H., Msi",Korupsi tergolong ke dalam kejahatan yang luar...,Korupsi tergolong ke dalam kejahatan yang luar...
...,...,...,...,...,...,...,...,...,...
137,Ilmu Pendidikan,30,Pgpaud,Pengaruh Strategi Meaningful Instructional Des...,Siti Herlinah Wifroh,"Yulias Wulani Fajar, M.Pd","Titin Faridatun Nisa', S.Pd., M.Pd",Tujuan penelitian ini untuk mengetahui pengaru...,The purpose of this study is to reveal the inf...
138,Ilmu Pendidikan,38,Pendidikan Profesi Guru,Tidak ada jurnal,None,None,None,None,None
139,Mbkm,99,Mbkm,Tidak ada jurnal,None,None,None,None,None
140,Pascasarjana,39,Magister Pendidikan Dasar,Tidak ada jurnal,None,None,None,None,None
